In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
# Import the time module
import time

# Record the start time
start_time = time.time()

# Specify the folder path where the 22 files are located
folder_path = ''

# Initialize an empty list to store all the dataframes
all_dfs = []

# Loop through each file in the specified folder
for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(folder_path, filename)
        
        # Read the Excel file into a dataframe
        df = pd.read_excel(file_path)
        
        # Convert 'iranidate' to datetime and handle missing values
        df['iranidate'] = pd.to_datetime(df['iranidate'], errors='coerce')
        
        def convert_to_numeric_date(date):
            base_date = datetime(2000, 1, 1)
            return (date - base_date).days
        df['iranidate'] = df['iranidate'].apply(convert_to_numeric_date)
        
        # Encode 'node' column
        label_encoder = LabelEncoder()
        df['node_encoded'] = label_encoder.fit_transform(df['node'])
        
        # Handling NaN values by finding the nearest non-empty cells in the same column
        for col in ['CO', 'O3', 'SO2', 'PM10', 'PM2.5', 'AQI']:
            for idx, value in df[col].items():
                if pd.isnull(value):  
                    upper_cell = df[col].iloc[:idx].last_valid_index()  
                    lower_cell = df[col].iloc[idx + 1:].first_valid_index()  
                    if upper_cell is not None and lower_cell is not None:  
                        avg = (df.at[upper_cell, col] + df.at[lower_cell, col]) / 2  
                        df.at[idx, col] = avg  
        
        # Append the modified dataframe to the list
        all_dfs.append(df)

# Concatenate all dataframes into a single dataframe
df = pd.concat(all_dfs, ignore_index=True)

# Check sample threshold
your_threshold_value = 100
if len(df) < your_threshold_value:
    print("Insufficient samples in the dataset.")
else:
    output_columns = ['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5', 'AQI', 'Highest tempreture: 12pm', 'Wind:km/h']
    unique_nodes = df['node_encoded'].unique()  

    # Dictionary to store weighted predictions for each node and column
    weighted_predictions = {}
    
    lstm_model = Sequential([
        LSTM(50, activation='relu', input_shape=(2, 1)),
        Dense(1)
    ])
    lstm_model.compile(optimizer='adam', loss='mse')
    xgb_model_2 = xgb.XGBRegressor(colsample_bytree=0.8, learning_rate=0.3, max_depth=10, alpha=10, n_estimators=200)
    xgb_model_3 = xgb.XGBRegressor(colsample_bytree=0.8, learning_rate=0.3, max_depth=10, alpha=10, n_estimators=200)

    for node_value in unique_nodes:
        node_df = df[df['node_encoded'] == node_value]  
        
        # Dictionary to store predictions and true values for each column
        columns_data = {}

        for output_column in output_columns:
            if node_df[output_column].isnull().any():
                print(f"NaN values found in {output_column} for Node {node_value}. Handle missing values before modeling.")
            else:
                # Modify X to include the 'node' column along with 'iranidate'
                X = node_df[['iranidate', 'node_encoded']]
                Y = node_df[[output_column]]  
                X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

                # Reshape the input data for LSTM
                X_train_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
                X_test_lstm = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

                lstm_model.fit(X_train_lstm, Y_train, epochs=50, batch_size=32, verbose=0)

                # Predict using the LSTM model
                Y1_pred_lstm = lstm_model.predict(X_test_lstm)

                # Flatten the LSTM predictions for input to XGBoost model
                Y1_pred_lstm_flat = Y1_pred_lstm.flatten()

                xgb_model_2.fit(np.concatenate((X_test, Y1_pred_lstm_flat.reshape(-1, 1)), axis=1), Y_test)
                Y2_pred_xgb = xgb_model_2.predict(np.concatenate((X_test, Y1_pred_lstm_flat.reshape(-1, 1)), axis=1))

                xgb_model_3.fit(np.concatenate((X_test, Y2_pred_xgb.reshape(-1, 1)), axis=1), Y_test)
                Y3_pred_xgb = xgb_model_3.predict(np.concatenate((X_test, Y2_pred_xgb.reshape(-1, 1)), axis=1))

                # Calculate errors for the third XGBoost model
                mse_lstm_1 = mean_squared_error(Y_test, Y1_pred_lstm)
                mse_xgb_2 = mean_squared_error(Y_test, Y2_pred_xgb)
                mse_xgb_3 = mean_squared_error(Y_test, Y3_pred_xgb)
                rmse_xgb_3 = np.sqrt(mse_xgb_3)
                r2_xgb_3 = r2_score(Y_test, Y3_pred_xgb)
                evs_xgb_3 = explained_variance_score(Y_test, Y3_pred_xgb)

                # Store predictions and true values for each column in the dictionary
                columns_data[output_column] = (Y3_pred_xgb, Y_test)

                # Output results for the local model with weighted predictions
                print(f"Local Model Results for Node {node_value} - {output_column} with Weighted Predictions:")
                print(f"MSE for LSTM_1: {mse_lstm_1}")
                print(f"MSE for XGB_2: {mse_xgb_2}")
                print(f"MSE for XGB_3: {mse_xgb_3}")
                print(f"RMSE for XGB_3: {rmse_xgb_3}")
                print(f"R2 for XGB_3: {r2_xgb_3}")
                print(f"EVS for XGB_3: {evs_xgb_3}")

                # Store weighted predictions for each node and column
                if node_value not in weighted_predictions:
                    weighted_predictions[node_value] = {output_column: (Y3_pred_xgb, Y_test)}
                else:
                    weighted_predictions[node_value][output_column] = (Y3_pred_xgb, Y_test)

# Aggregate predictions for the global model using weighted average
global_X_test = []
global_Y_test = []

for node, columns in weighted_predictions.items():
    for column, (node_prediction, Y_test) in columns.items():
        global_X_test.extend(node_prediction * r2_xgb_3)  # Apply weights using the R2 score from the LSTM model
        global_Y_test.extend(Y_test.values)

# Create and train the global model (Random Forest)
global_model = RandomForestRegressor(random_state=42)
global_model.fit(np.array(global_X_test).reshape(-1, 1), global_Y_test)  # Reshape for RF

# Predict using the global model
global_Y_pred = global_model.predict(np.array(global_X_test).reshape(-1, 1))
global_mse = mean_squared_error(global_Y_test, global_Y_pred)

# Print results for the global model
print("Global Model Results:")
print(f"Mean Squared Error (MSE) for Global Model: {global_mse}")

# Record the end time
end_time = time.time()

# Calculate the duration of the program execution
duration_seconds = end_time - start_time

# Print the duration of the program execution
print(f"Duration of the program execution: {duration_seconds} seconds")


2024-02-27 20:56:25.911898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


51/51 [==============================] - 0s 2ms/step
Local Model Results for Node 0 - CO with Weighted Predictions:
MSE for LSTM_1: 206.41255059480218
MSE for XGB_2: 89.05519876455325
MSE for XGB_3: 87.89050290880817
RMSE for XGB_3: 9.374993488467508
R2 for XGB_3: 0.49921073109585967
EVS for XGB_3: 0.49921075892015265
51/51 [==============================] - 0s 1ms/step
Local Model Results for Node 0 - O3 with Weighted Predictions:
MSE for LSTM_1: 1268.2620245925903
MSE for XGB_2: 234.2920838881427
MSE for XGB_3: 233.28963815136527
RMSE for XGB_3: 15.273821988990354
R2 for XGB_3: 0.8181785668628083
EVS for XGB_3: 0.8181785668633166
51/51 [==============================] - 0s 1ms/step
Local Model Results for Node 0 - NO2 with Weighted Predictions:
MSE for LSTM_1: 547.8860450386782
MSE for XGB_2: 337.803022436783
MSE for XGB_3: 336.6361423038784
RMSE for XGB_3: 18.347646778371292
R2 for XGB_3: 0.3805246393230256
EVS for XGB_3: 0.3805246399508958
51/51 [==============================] - 0

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Global Model Results:
Mean Squared Error (MSE) for Global Model: 195.15350801074726
Duration of the program execution: 158.62399792671204 seconds
